In [ ]:
import sys
import os

sys.path.append('..')

from interpretDistill.fourierDistill import *
from interpretDistill.binaryTransformer import *

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

X, y = load_iris(as_frame = True, return_X_y = True)
X.columns = X.columns.str.replace(' ', '_')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [ ]:
bt_bit = BinaryTransformer(depth = 4, bit = True)
bt_bin = BinaryTransformer(depth = 4, bit = False)

In [ ]:
X_train_bit = bt_bit.fit_and_transform(X_train, y_train)
X_val_bit = bt_bit.transform(X_val)
X_test_bit = bt_bit.transform(X_test)

In [ ]:
X_train_bin = bt_bin.fit_and_transform(X_train, y_train)
X_val_bin = bt_bin.transform(X_val)
X_test_bin = bt_bin.transform(X_test)

In [ ]:
print(f'bit number of features: {X_train_bit.shape[1]}')
print(f'bin number of features: {X_train_bin.shape[1]}')

In [ ]:
rf_orig = RandomForestRegressor(max_depth = 6)
rf_bit = RandomForestRegressor(max_depth = 6)
rf_bin = RandomForestRegressor(max_depth = 6)

In [ ]:
rf_orig.fit(X_train, y_train)
rf_bit.fit(X_train_bit, y_train)
rf_bin.fit(X_train_bin, y_train)

In [ ]:
print(f'[orig] train MSE: {mean_squared_error(rf_orig.predict(X_train),y_train)}, val MSE: {mean_squared_error(rf_orig.predict(X_val),y_val)}, test MSE: {mean_squared_error(rf_orig.predict(X_test),y_test)}')
print(f'[bit] train MSE: {mean_squared_error(rf_bit.predict(X_train_bit),y_train)}, val MSE: {mean_squared_error(rf_bit.predict(X_val_bit),y_val)}, test MSE: {mean_squared_error(rf_bit.predict(X_test_bit),y_test)}')
print(f'[bin] train MSE: {mean_squared_error(rf_bin.predict(X_train_bin),y_train)}, val MSE: {mean_squared_error(rf_bin.predict(X_val_bin),y_val)}, test MSE: {mean_squared_error(rf_bin.predict(X_test_bin),y_test)}')

In [ ]:
print(f'[orig RF] train R2: {r2_score(rf_orig.predict(X_train),y_train)}, val R2: {r2_score(rf_orig.predict(X_val),y_val)}, test R2: {r2_score(rf_orig.predict(X_test),y_test)}')
print(f'[bit RF] train R2: {r2_score(rf_bit.predict(X_train_bit),y_train)}, val R2: {r2_score(rf_bit.predict(X_val_bit),y_val)}, test R2: {r2_score(rf_bit.predict(X_test_bit),y_test)}')
print(f'[bin RF] train R2: {r2_score(rf_bin.predict(X_train_bin),y_train)}, val R2: {r2_score(rf_bin.predict(X_val_bin),y_val)}, test R2: {r2_score(rf_bin.predict(X_test_bin),y_test)}')

In [ ]:
ftd_bit = FTDistillCV(size_interactions = 3)
ftd_bin = FTDistillCV(size_interactions = 3)

In [ ]:
ftd_bit.fit(X_val_bit, rf_orig.predict(X_val))
ftd_bin.fit(X_val_bin, rf_orig.predict(X_val), bt_bin.no_interaction)

In [ ]:
sum(ftd_bit.regression_model.coef_ != 0), sum(ftd_bin.regression_model.coef_ != 0)

In [ ]:
len(ftd_bit.regression_model.coef_), len(ftd_bin.regression_model.coef_)

In [ ]:
print(f'[bit FTD, true y] train MSE: {mean_squared_error(ftd_bit.predict(X_train_bit),y_train)}, val MSE: {mean_squared_error(ftd_bit.predict(X_val_bit),y_val)}, test MSE: {mean_squared_error(ftd_bit.predict(X_test_bit),y_test)}')
print(f'[bit FTD, RF y] train MSE: {mean_squared_error(ftd_bit.predict(X_train_bit),rf_bit.predict(X_train_bit))}, val MSE: {mean_squared_error(ftd_bit.predict(X_val_bit),rf_bit.predict(X_val_bit))}, test MSE: {mean_squared_error(ftd_bit.predict(X_test_bit),rf_bit.predict(X_test_bit))}')
print(f'[bin FTD, true y] train MSE: {mean_squared_error(ftd_bin.predict(X_train_bin),y_train)}, val MSE: {mean_squared_error(ftd_bin.predict(X_val_bin),y_val)}, test MSE: {mean_squared_error(ftd_bin.predict(X_test_bin),y_test)}')
print(f'[bin FTD, RF y] train MSE: {mean_squared_error(ftd_bin.predict(X_train_bin),rf_bin.predict(X_train_bin))}, val MSE: {mean_squared_error(ftd_bin.predict(X_val_bin),rf_bin.predict(X_val_bin))}, test MSE: {mean_squared_error(ftd_bin.predict(X_test_bin),rf_bin.predict(X_test_bin))}')

In [ ]:
print(f'[bit FTD, true y] train R2: {r2_score(ftd_bit.predict(X_train_bit),y_train)}, val R2: {r2_score(ftd_bit.predict(X_val_bit),y_val)}, test R2: {r2_score(ftd_bit.predict(X_test_bit),y_test)}')
print(f'[bit FTD, RF y] train R2: {r2_score(ftd_bit.predict(X_train_bit),rf_orig.predict(X_train))}, val R2: {r2_score(ftd_bit.predict(X_val_bit),rf_orig.predict(X_val))}, test R2: {r2_score(ftd_bit.predict(X_test_bit),rf_orig.predict(X_test))}')
print(f'[bin FTD, true y] train R2: {r2_score(ftd_bin.predict(X_train_bin),y_train)}, val R2: {r2_score(ftd_bin.predict(X_val_bin),y_val)}, test R2: {r2_score(ftd_bin.predict(X_test_bin),y_test)}')
print(f'[bin FTD, RF y] train R2: {r2_score(ftd_bin.predict(X_train_bin),rf_orig.predict(X_train))}, val R2: {r2_score(ftd_bin.predict(X_val_bin),rf_orig.predict(X_val))}, test R2: {r2_score(ftd_bin.predict(X_test_bin),rf_orig.predict(X_test))}')

In [ ]:
from itertools import compress

sorted([i for i in zip(list(compress(ftd_bit.features, ftd_bit.regression_model.coef_ != 0)), list(compress(ftd_bit.regression_model.coef_, ftd_bit.regression_model.coef_ != 0)))], key = lambda x: abs(x[1]))[-3:]
#sorted([i for i in zip(list(compress(ftd_bin.features, ftd_bin.regression_model.coef_ != 0)), list(compress(ftd_bin.regression_model.coef_, ftd_bin.regression_model.coef_ != 0)))], key = lambda x: abs(x[1]))

In [ ]:
kill

In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
solar_flare = fetch_ucirepo(id=89) 
  
# data (as pandas dataframes) 
X = solar_flare.data.features 
y = solar_flare.data.targets 
  
# metadata 
X

In [ ]:
import matplotlib.pyplot as plt